In [1]:
#XGBoost on scaled data

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_excel('/Users/didyvanbelkom/Documents/02 MSc DSS/THESIS CODE/Jupiter Notebook/df_complete_2.xlsx')

In [4]:
df.head()

,Customer ID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,TotalPrice,Top 10% spend
0,12346,530,11,372.86,2,5,2,25,77183.60,1
1,12347,368,2,1323.32,5,2,4,52,4310.00,1
2,12348,439,1,222.16,2,1,1,21,1797.24,0
3,12349,408,3,2671.14,3,3,5,33,1757.55,0
4,12352,376,2,343.80,5,2,2,52,2506.04,0


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# import required libraries for clustering
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

In [6]:
km = KMeans(n_clusters = 2)
km

KMeans(n_clusters=2)

In [7]:
np.random.seed(123)

In [8]:
y_predicted = km.fit_predict(df[['recency_score', 'frequency_score', 'monetary_score']])
y_predicted

array([0, 1, 0, ..., 0, 1, 1], dtype=int32)

In [9]:
df['cluster'] = y_predicted
df.head()

,Customer ID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,TotalPrice,Top 10% spend,cluster
0,12346,530,11,372.86,2,5,2,25,77183.60,1,0
1,12347,368,2,1323.32,5,2,4,52,4310.00,1,1
2,12348,439,1,222.16,2,1,1,21,1797.24,0,0
3,12349,408,3,2671.14,3,3,5,33,1757.55,0,1
4,12352,376,2,343.80,5,2,2,52,2506.04,0,0


In [10]:
df.value_counts()

Customer ID  recency  frequency  monetary  recency_score  frequency_score  monetary_score  RFM_SCORE  TotalPrice  Top 10% spend  cluster
18287        383      4          2345.71   4              4                5               44         1837.28     0              1          1
14403        590      2          404.70    1              2                2               12         1475.81     0              0          1
14415        380      10         6191.32   5              5                5               55         5900.28     1              1          1
14414        375      1          902.35    5              1                3               51         1496.98     0              0          1
14413        411      3          647.67    3              3                3               33         122.70      0              0          1
                                                                                                                                           ..
16325      

In [11]:
test = df.groupby(['cluster', 'Top 10% spend']).count()

In [12]:
print(test['Customer ID'])

cluster  Top 10% spend
0        0                1179
         1                  19
1        0                1211
         1                 363
Name: Customer ID, dtype: int64


In [13]:
df_test = df.loc[df['cluster'] == 1]

In [14]:
df_test.describe()

,Customer ID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,TotalPrice,Top 10% spend,cluster
count,1574.000000,1574.000000,1574.000000,1574.000000,1574.000000,1574.000000,1574.000000,1574.000000,1574.000000,1574.000000,1574.0
mean,15419.355781,394.308132,8.810673,4505.397172,4.080686,4.380559,4.344981,45.187421,3974.167167,0.230623,1.0
std,1685.567858,31.976028,12.104430,14335.652501,1.016943,0.712724,0.742701,10.307038,13692.810840,0.421365,0.0
min,12347.000000,366.000000,1.000000,329.480000,1.000000,1.000000,2.000000,14.000000,15.000000,0.000000,1.0
25%,14008.000000,373.000000,4.000000,1283.885000,3.000000,4.000000,4.000000,35.000000,665.432500,0.000000,1.0
50%,15481.000000,382.000000,6.000000,2137.860000,4.000000,5.000000,4.000000,45.000000,1600.190000,0.000000,1.0
75%,16890.500000,406.500000,9.000000,3865.085000,5.000000,5.000000,5.000000,55.000000,3380.275000,0.000000,1.0
max,18287.000000,594.000000,205.000000,349164.350000,5.000000,5.000000,5.000000,55.000000,280206.020000,1.000000,1.0


In [15]:
df_test = df_test.drop(columns=['Customer ID', 'TotalPrice', 'cluster', 'recency', 'frequency', 'monetary', 'RFM_SCORE'])

In [16]:
df_test.head()

,recency_score,frequency_score,monetary_score,Top 10% spend
1,5,2,4,1
3,3,3,5,0
7,4,3,5,0
8,4,2,5,1
9,5,3,5,0


In [17]:
df_test['Top 10% spend'].value_counts()

0    1211
1     363
Name: Top 10% spend, dtype: int64

In [18]:
X = df_test.drop(columns = ['Top 10% spend'])
y = df_test['Top 10% spend']

In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=123)

In [20]:
#XGBoost on scaled data 

In [21]:
import xgboost as xgb
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', missing = None, random_state = 123, use_label_encoder=False)

In [22]:
clf_xgb.fit(X_train, 
            y_train,
            verbose = True,
            early_stopping_rounds = 10,
            eval_metric = 'aucpr',
            eval_set = [[X_test, y_test]])

[0]	validation_0-aucpr:0.47885
[1]	validation_0-aucpr:0.48314
[2]	validation_0-aucpr:0.48323
[3]	validation_0-aucpr:0.48402
[4]	validation_0-aucpr:0.48722
[5]	validation_0-aucpr:0.48722
[6]	validation_0-aucpr:0.48722
[7]	validation_0-aucpr:0.48717
[8]	validation_0-aucpr:0.48854
[9]	validation_0-aucpr:0.47565
[10]	validation_0-aucpr:0.47597
[11]	validation_0-aucpr:0.47636
[12]	validation_0-aucpr:0.47636
[13]	validation_0-aucpr:0.47630
[14]	validation_0-aucpr:0.47636
[15]	validation_0-aucpr:0.47690
[16]	validation_0-aucpr:0.47690
[17]	validation_0-aucpr:0.47690
[18]	validation_0-aucpr:0.47734


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [23]:
xgb_preds = clf_xgb.predict(X_test)

In [24]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, xgb_preds)
cnf_matrix

array([[322,  48],
       [ 49,  54]])

In [25]:
print("Accuracy:",metrics.accuracy_score(y_test, xgb_preds))
print("Precision:",metrics.precision_score(y_test, xgb_preds))
print("Recall:",metrics.recall_score(y_test, xgb_preds))

Accuracy: 0.7949260042283298
Precision: 0.5294117647058824
Recall: 0.5242718446601942


In [26]:
y_pred_proba = clf_xgb.predict_proba(X_test)[::,1]
auc = metrics.roc_auc_score(y_test, y_pred_proba)

In [27]:
print(auc)

0.8221726580949883


In [28]:
f1_xgb = metrics.f1_score(y_test, xgb_preds)
print(f1_xgb)

0.5268292682926828
